In [27]:
import json
import pandas as pd
with open("data/fromgithub2.json","r") as f:
    df = pd.read_json(f)
    # posts = json.loads(f.read())

df = df[["title","time","url"]]
df["time"] = df["time"].astype("datetime64[s]",)
df["visited"] = 0
# df.to_csv("data/fromgithub2.csv")
df

,title,time,url,visited
0,Jraft – the best blogging platform ever,2020-11-16 02:12:45,http://www.tryjraft.com,0
1,"California, Love It and Leave It",2020-11-16 01:33:38,https://www.wsj.com/articles/california-love-i...,0
2,"SpaceX Starlink has some hiccups as expected, ...",2020-11-16 00:56:03,https://arstechnica.com/information-technology...,0
3,SpaceX Crew-1 [video],2020-11-16 00:23:38,https://www.youtube.com/watch?v=bnChQbxLkkI&re...,0
4,The Treehouse Is De-Plaformed,2020-11-15 23:28:32,https://theconservativetreehouse.com/2020/11/1...,0
...,...,...,...,...
1487,U.S. Feds Seized Nearly $1B in Bitcoin from Wa...,2020-11-05 18:22:07,https://www.vice.com/en/article/akdgz8/us-feds...,0
1488,Facebook was used as a proxy by web scraping bots,2020-11-05 18:15:42,https://datadome.co/bot-detection/how-facebook...,0
1489,How is Visual Basic still ranked #6 programmin...,2020-11-05 18:06:53,https://www.tiobe.com/tiobe-index/,0
1490,Confessions of a voter fraud: I was a master a...,2020-11-05 17:48:36,https://nypost.com/2020/08/29/political-inside...,0


In [3]:
from datetime import datetime

# features_df = df[["title","time","kids"]]
# titles = df.loc[df["title"].isna()==False,"title"]
df.dropna(subset=["title"])
titles = df["title"]

# features_df.loc[:,"kids"] = features_df["kids"].apply(lambda x: len(x))
# features_df["time"] = features_df["time"].astype('datetime64[s]')
titles.head()

0    We Don’t Need Elections to Figure Out What Peo...
1                Hegel and the History of Human Nature
2                       Ask HN: What's Life All About?
3        Ask HN: What do Stoics think of Ecclesiastes?
4                        The Problem of Centralisation
Name: title, dtype: object

In [3]:
import numpy as np
from transformers import DistilBertTokenizer, DistilBertModel 
from transformers import BertTokenizer, BertModel

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")
# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# model = BertModel.from_pretrained("bert-base-uncased")


tokenized = titles.apply(
    (lambda x: tokenizer.encode(x, add_special_tokens=True)))

max_len = max([len(row) for row in tokenized])
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
attention_mask = np.where(padded!=0 , 1, 0)

padded

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


array([[  101,  2057,  2123, ...,     0,     0,     0],
       [  101,  2002, 12439, ...,     0,     0,     0],
       [  101,  3198,  1044, ...,     0,     0,     0],
       ...,
       [  101,  2017,  2453, ...,     0,     0,     0],
       [  101, 19657,  5448, ...,     0,     0,     0],
       [  101, 18546,  9832, ...,     0,     0,     0]])

In [4]:
import torch
padded = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)
attention_mask, attention_mask.shape

(tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 torch.Size([600, 31]))

In [5]:
output = model(padded, attention_mask=attention_mask)
output

BaseModelOutput(last_hidden_state=tensor([[[ 0.0771,  0.0182, -0.0643,  ..., -0.1913,  0.1890,  0.3527],
         [ 0.4723, -0.0055, -0.2808,  ..., -0.2538,  0.3928, -0.1306],
         [-0.0460,  0.2772,  0.3011,  ..., -0.5342,  0.0617, -0.2683],
         ...,
         [ 0.2057, -0.2744, -0.1650,  ..., -0.0303, -0.1430,  0.0469],
         [ 0.3543, -0.0234,  0.0338,  ..., -0.1402,  0.0170, -0.3511],
         [ 0.1917,  0.1626, -0.0960,  ..., -0.2867, -0.0862, -0.2281]],

        [[-0.3856,  0.0923, -0.5885,  ..., -0.1052,  0.3748,  0.5154],
         [ 0.0447,  0.0093, -0.0174,  ...,  0.0485,  0.4887, -0.2297],
         [-0.0884,  0.6354, -0.4877,  ..., -0.2014, -0.1107,  0.6105],
         ...,
         [-0.0048,  0.2007, -0.2539,  ...,  0.2085,  0.0093,  0.3343],
         [-0.2145,  0.2177, -0.1630,  ..., -0.0419,  0.2994,  0.3233],
         [-0.2263,  0.3092, -0.1494,  ..., -0.1419,  0.2361,  0.1098]],

        [[ 0.0413, -0.0265, -0.1379,  ...,  0.0640,  0.3722,  0.3841],
         [ 

In [6]:
features = output[0][:,0,:].detach().numpy()
print(features.shape)
features

(600, 768)


array([[ 0.07705708,  0.01822153, -0.06425738, ..., -0.19133918,
         0.18901303,  0.35266334],
       [-0.38561448,  0.09233379, -0.58854973, ..., -0.10521927,
         0.37476808,  0.51544446],
       [ 0.0412614 , -0.02650118, -0.13785702, ...,  0.06401408,
         0.37219882,  0.384129  ],
       ...,
       [-0.20407574, -0.20291403, -0.18362422, ..., -0.1841933 ,
         0.12020883,  0.4585222 ],
       [-0.09072461, -0.26008007, -0.01645303, ..., -0.05409419,
         0.5282614 ,  0.41041008],
       [-0.4049973 , -0.33266595,  0.17361143, ..., -0.13655995,
         0.23092951,  0.30166227]], dtype=float32)

In [7]:
# fill dummy data
y = [1 if i%2==0 else 0 for i in range(len(features))]
y = np.array(y)

In [8]:
from sklearn.model_selection import train_test_split
RANDOM_STATE = 42

X_train, X_test, y_train, y_test = train_test_split(
    features, y, test_size=0.33, random_state=RANDOM_STATE)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((402, 768), (198, 768), (402,), (198,))

In [9]:
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score

print("DUMMY")
clf = DummyClassifier(random_state=RANDOM_STATE)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)
# cross_val_score(clf, X_test, y_test)

DUMMY


0.45454545454545453

In [10]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import cross_val_score

print("LINEAR, LOGISTIC")
clf = LinearRegression()
# clf = LogisticRegression(random_state=RANDOM_STATE)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)
# cross_val_score(clf, X_test, y_test)

LINEAR, LOGISTIC


-8.588361918695732

In [11]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score

print("NAIVE BAYES")
clf = GaussianNB()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)
# cross_val_score(clf, X_test, y_test)

NAIVE BAYES


0.5202020202020202

In [12]:
"""
##############################33333
# column tranformer for preprocessing. TRASH.ed.
##############################33333
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

from sklearn.compose import ColumnTransformer

# # titles = df["title"]
# # titles
# class TextTransformer:
#     def __init__(self, tokenizer, model):
#         self.tokenizer = tokenizer
#         self.model = model
    
#     def fit(self, X, y=None):
#         return self

#     def transform(self, X):
#         tokenized = titles.apply(
#             (lambda x: tokenizer.encode(x, add_special_tokens=True)))
#         max_len = max([len(row) for row in tokenized])
#         padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
#         attention_mask = np.where(padded!=0 , 1, 0)
#         padded = torch.tensor(padded)
#         attention_mask = torch.tensor(attention_mask)
#         output = model(padded, attention_mask=attention_mask)
#         features = output[0][:,0,:].detach().numpy()
#         return features


# preprocessor = ColumnTransformer([
#     ("text_trasnform", TextTransformer, ["title"])
#     ],
#     verbose=True,
# )

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
# pipeline = Pipeline([
#     ("column_transformer",preprocessor),
#     ("classifier", LogisticRegression()),
# ])

# # 36153496,The Meta Quest 3 is a $499 mixed reality headset with full-color passthrough (engadget.com),8,2.0,0
# # 36151132,People living near former atomic weapon plant in WA have increased cancer rates (nih.gov),61,17.0,0
# # 36149690,Maryland License Plates Now Inadvertently Advertising Filipino Online Casino (vice.com),171,85.0,0
# # 36135250,Sun Remarketing Catalog Summer 1988 (archive.org),66,23.0,0
# # 36149375,EPYC 7002 CPUs may hang after 1042 days of uptime (reddit.com),130,82.0,0


from sklearn.feature_extraction.text import TfidfVectorizer

# Create a pipeline
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

text_transformer = Pipeline([
    ('tokenizer', tokenizer),
    ('vectorizer', TfidfVectorizer(tokenizer=tokenizer.tokenize)),
])

preprocessor = ColumnTransformer([
    ('text', text_transformer, 'title')
])

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])


y = [1 if i%2==0 else 0 for i in range(len(features))]
y = np.array(y)

from sklearn.model_selection import train_test_split
RANDOM_STATE = 42
X_train, X_test, y_train, y_test = train_test_split(
    df[["title"]], y, test_size=0.2, random_state=RANDOM_STATE
)

print(X_train.shape,X_test.shape, y_train.shape, y_test.shape)
print(y_train)

pipeline.fit(X_train, y_train)
# pipeline.predict(X_test)

# pipeline.transform(["A very clickbaity title from Tesla, Apple, Google."])
"""

'\n##############################33333\n# column tranformer for preprocessing. TRASH.ed.\n##############################33333\ntokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")\nmodel = DistilBertModel.from_pretrained("distilbert-base-uncased")\n\nfrom sklearn.compose import ColumnTransformer\n\n# # titles = df["title"]\n# # titles\n# class TextTransformer:\n#     def __init__(self, tokenizer, model):\n#         self.tokenizer = tokenizer\n#         self.model = model\n    \n#     def fit(self, X, y=None):\n#         return self\n\n#     def transform(self, X):\n#         tokenized = titles.apply(\n#             (lambda x: tokenizer.encode(x, add_special_tokens=True)))\n#         max_len = max([len(row) for row in tokenized])\n#         padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])\n#         attention_mask = np.where(padded!=0 , 1, 0)\n#         padded = torch.tensor(padded)\n#         attention_mask = torch.tensor(attention_mask)\n# 